In [1]:
#This part of the 5 parts tutorial shows you how to topic modelling! 
#Our data-set is a list of talks and abstracts from the CCC conference https://gitlab.com/maxigas/cccongresstalks/
#The data is available as .csv file format, we'll play with the data to show what topics are being focused on 
#    using Latent Dirichlet Allocation (LDA).

#this is to avoid showing warning, comment out otherwise.
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import matplotlib.pyplot as plt
import seaborn as sns
import mglearn
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#Reading the 2016.csv CCC talks, which is stored as a CSV file in the csvs directory.
#notice the delimiter is not a comma, check your files first.
File2016_df = pd.read_csv("csvs/2016.csv", delimiter='|', header=0)
print('Number of titles: {:,}\n'.format(File2016_df.shape[0]))
#print a sample of 5 rows.
File2016_df.sample(5)

Number of titles: 149



,year,congress,title,abstract,link,tags
27,2016,33C3,Space making/space shaping: Wider die automati...,What are the politics and aesthetics of mappin...,https://events.ccc.de/congress/2016/Fahrplan/s...,NaN
138,2016,33C3,Mass Surveillance through Computational Lingui...,Even though the Snowden revelations for the fi...,https://events.ccc.de/congress/2016/Fahrplan/s...,NaN
11,2016,33C3,Console hackz 2016: Ein Geheimdienst als Zeuge...,"Last year, we demonstrated Linux running on th...",https://events.ccc.de/congress/2016/Fahrplan/s...,NaN
29,2016,33C3,Routerzwang und Funkabschottung: A proposal fo...,Nach drei Jahren wurde endlich die nutzerunfre...,https://events.ccc.de/congress/2016/Fahrplan/s...,NaN
25,2016,33C3,Building a high throughput low-latency PCIe ba...,Software Defined Radios (SDRs) became a mainst...,https://events.ccc.de/congress/2016/Fahrplan/s...,NaN


In [3]:
#reading titles (title is a column in the csv file as shown in the sample above)

#loop through the titles and store them in a dictionary.
title_dict = {}

# loop through titles
for i in range(len(File2016_df)):
    if File2016_df["title"][i] in title_dict.keys():
        title_dict[File2016_df["title"][i]] += 1
    else:
        title_dict.setdefault(File2016_df["title"][i], 1)

In [4]:
#we do the same thing for the abstract column

abstract_dict = {}

# classify that the article has recommends
for i in range(len(File2016_df)):
    if File2016_df["abstract"][i] in abstract_dict.keys():
        abstract_dict[File2016_df["abstract"][i]] += 1
    else:
        abstract_dict.setdefault(File2016_df["abstract"][i], 1)

abstract_dict = [x for x in abstract_dict if str(x) != 'nan']#some talks have no abstracts

In [5]:
#spacy tokenizer
#a spacy method to extract clean tokens from text
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    all_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            all_tokens.append('URL')
        elif token.orth_.startswith('@'):
            all_tokens.append('SCREEN_NAME')
        else:
            all_tokens.append(token.lower_)
    return all_tokens

In [6]:
#NLTK’s Wordnet used to find the meanings of words, synonyms, antonyms, and more. 
#I import English and German stop words lists since the talks contain German text, especially in the old days.

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
de_stop = set(nltk.corpus.stopwords.words('german'))

#preprocess text by removing stop words and keeping words with more than 4 letters (my choice you can change it)
def preprocess_text(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop or token in de_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elhaj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elhaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#I convert the dictionary into a list with clean pre-processed text
title_text_data = []
print(type(title_text_data))
counter = 0
for title in title_dict:
    counter = counter + 1
    title_tokens = preprocess_text(title.replace('attackz', 'attacks').replace('hackz', 'hack').replace('securityz', 'security'))#I do some replacing for German-style words!
    if (counter < 11):#this is just to print the first 10 lines
        print(title_tokens)
    title_text_data.append(title_tokens)

<class 'list'>
['opening', 'ceremony', 'infrastructure', 'people', 'behind', 'drone', 'killing']
['global', 'assassination', 'crypto', 'fight', 'apple']
['fight', 'encryption']
['bootstrap', 'slightly', 'security', 'laptop', 'local', 'computer', 'outside', 'jurisdiction']
['enforcement', 'planet', 'security']
['money', 'technical', 'teardown', 'pegasus', 'malware', 'trident', 'security', 'chain']
['pegasus', 'internals', 'automation', 'dependency', 'aviation', 'accident']
['stink']
['dieselgate', 'later', 'review', 'dieselgate']
['software', 'define', 'emission', 'add', 'device', 'sometimes', 'smart']


In [8]:
#The same is done to the abstracts dictionary
abstract_text_data = []
counter2 = 0
for abstract in abstract_dict:
    counter2 = counter2 + 1
    abstract_tokens = preprocess_text(abstract.replace('attackz', 'attacks').replace('hackz', 'hack').replace('securityz', 'security'))
    if (counter2 < 11):
        print(abstract_tokens)
    abstract_text_data.append(abstract_tokens)

['force', 'comms', 'bomb', 'technician', 'insight', 'invisible', 'network', 'governing', 'military', 'drone', 'quest', 'accountability']
['strong', 'communications', 'encryption', 'device', 'encryption', 'legal', 'jurisdiction', 'today', 'remain', 'widely', 'available', 'software', 'programers', 'hardware', 'producer', 'increasingly', 'pressure', 'enforcement', 'policy', 'maker', 'around', 'world', 'include', 'call', 'backdoor', 'encryption', 'product', 'lecture', 'provide', 'state', 'move', 'detailing', 'happen', 'fight', 'apple', 'bernardino', 'current', 'proposal', 'weaken', 'encryption', 'covering', 'propose', 'recently', 'enact', 'discus', 'extra', 'legal', 'pressure', 'place', 'company', 'government', 'state', 'sponsor', 'malware', 'alternative', 'addition', 'weakening', 'software', 'finally', 'presentation', 'discus', 'possible', 'realistic', 'outcome', 'prediction', 'state', 'discus', 'fight', 'future', 'allow', 'security', 'communications', 'everyone']
['head', 'source', 'cust

In [9]:
#gensim for titles
#what is a Gensim? The next important object you need to familiarize with in order to work in gensim is the Corpus (a Bag of Words). That is, it is a corpus object that contains the word id and its frequency in each document. You can think of it as gensim's equivalent of a Document-Term matrix
#The main function is doc2bow , which converts a collection of words to its bag-of-words representation: a list of (word_id, word_frequency) 2-tuples
from gensim import corpora
title_dictionary = corpora.Dictionary(title_text_data)
title_corpus = [title_dictionary.doc2bow(text) for text in title_text_data]

import pickle
#Pickle is needed to serialize gensim into a stream of bytes to be stored as a file.
#gensim and pkl files are stored in the 'models' directory.
pickle.dump(title_corpus, open('models/title_corpus.pkl', 'wb'))
title_dictionary.save('models/title_dictionary.gensim')

In [10]:
#gensim for abstracts
from gensim import corpora
abstract_dictionary = corpora.Dictionary(abstract_text_data)
abstract_corpus = [abstract_dictionary.doc2bow(text) for text in abstract_text_data]

import pickle
pickle.dump(abstract_corpus, open('models/abstract_corpus.pkl', 'wb'))
abstract_dictionary.save('models/abstract_dictionary.gensim')

In [11]:
#title LDA topics (I went with 5 topics and 10 passes, you can update that to show more topics or have several passes)
# probability distribution
# Discover abstract “topics” that occur in a collection of documents (why some parts of the data are similar)
# What are the topics they are talking about in the conference talks
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(title_corpus, num_topics = NUM_TOPICS, id2word=title_dictionary, passes=10)
ldamodel.save('models/title_model.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.009*"knowledge" + 0.009*"network" + 0.009*"smart" + 0.009*"city"')
(1, '0.011*"world" + 0.011*"security" + 0.008*"hardware" + 0.008*"ceremony"')
(2, '0.024*"talks" + 0.021*"lightning" + 0.008*"attack" + 0.008*"internet"')
(3, '0.016*"security" + 0.016*"space" + 0.010*"privacy" + 0.007*"technology"')
(4, '0.014*"surveillance" + 0.011*"state" + 0.011*"security" + 0.007*"base"')


In [17]:
#Try a new sentence on networking security, 
#LDA will show that topic 3 has the highest probability assigned, and topic 4 has the second highest probability assigned. 
#The printed probabilities should add up to 1.
new_doc = 'Our new technique offers a full spectrum of data privacy and security services and technology'
new_doc = preprocess_text(new_doc)
new_doc_bow = title_dictionary.doc2bow(new_doc)
print(ldamodel.get_document_topics(new_doc_bow))

[(0, 0.040234867), (1, 0.040447272), (2, 0.040876847), (3, 0.49284607), (4, 0.38559493)]


In [18]:
#display and plot title topics
#Saliency: how important regardless of the term frequency (i.e. some terms are frequent but not informative)
#Paper and more details: http://vis.stanford.edu/files/2012-Termite-AVI.pdf
#Relevance: how relevant is a term to a certain topic
#Paper and more details: https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf
#The size of the bubble measures the importance of the topics, relative to the data.

dictionary = gensim.corpora.Dictionary.load('models/title_dictionary.gensim')
corpus = pickle.load(open('models/title_corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('models/title_model.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, title_corpus, title_dictionary, sort_topics=False)
pyLDAvis.display(lda_display, template_type='notebook')